<a href="https://colab.research.google.com/github/Ensama-cmd/CivilEngineeringAI/blob/main/Notebooks/03_integration_complete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Installation et import
!pip install ezdxf
import sys
sys.path.append('/content/CivilEngineeringAI/src')

from plan_generator import PlanGenerator
from transformers import pipeline
import spacy
import re
import json

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.8/5.8 MB 31.8 MB/s eta 0:00:00


ModuleNotFoundError: No module named 'plan_generator'

In [ ]:
# Chargement des modèles
print("Chargement des modèles NLP...")
nlp_spacy = spacy.load("fr_core_news_sm")

In [ ]:
# Fonction d'extraction (simplifiée pour Colab)
def extract_params(description):
    params = {
        "longueur": 10, # Valeurs par défaut
        "largeur": 8,
        "epaisseur_mur": 0.2
    }

    # Extraction de la surface pour déduire les dimensions
    surface_match = re.search(r'(\d+)\s*m²', description)
    if surface_match:
        surface = int(surface_match.group(1))
        # Ratio approximatif pour une maison
        params["longueur"] = round((surface * 1.2) ** 0.5, 1)
        params["largeur"] = round(surface / params["longueur"], 1)

    # Extraction de l'épaisseur des murs
    mur_match = re.search(r'(\d+)\s*cm', description)
    if mur_match:
        params["epaisseur_mur"] = int(mur_match.group(1)) / 100

    return params

In [2]:
# Pipeline complet
def generate_plan_from_description(description, filename="output_plan.dxf"):
    print(f"Traitement de la description: {description}")

    # Extraction des paramètres
    params = extract_params(description)
    print(f"Paramètres extraits: {params}")

    # Génération du plan
    generator = PlanGenerator()
    plan = generator.generate_simple_rectangular_plan(
        params["longueur"],
        params["largeur"],
        params["epaisseur_mur"]
    )

    # Sauvegarde
    generator.save_dxf(filename)

    return plan, params

In [3]:
# Test
description = "Maison de 120m² avec murs de 20cm"
plan, params = generate_plan_from_description(description)

Traitement de la description: Maison de 120m² avec murs de 20cm


NameError: name 'extract_params' is not defined

In [4]:

# Visualisation (optionnelle)
try:
    from google.colab import files
    files.download("output_plan.dxf")
except:
    print("Le fichier DXF a été généré. Téléchargez-le pour le visualiser.")

Le fichier DXF a été généré. Téléchargez-le pour le visualiser.


In [ ]:
# Import des nouveaux modules
from src.model3d_generator import Model3DGenerator
from src.structural_calculator import StructuralCalculator

In [ ]:
#  Génération du modèle 3D
print("\nGénération du modèle 3D...")
model3d_generator = Model3DGenerator()
model3d = model3d_generator.generate_simple_building(params)
model3d_generator.save_dxf("modele_3d.dxf")
print("Modèle 3D généré: modele_3d.dxf")

In [ ]:
# Calculs structurels
print("\nCalculs structurels en cours...")
structural_calculator = StructuralCalculator()
structural_report = structural_calculator.generate_structural_report(params)

print("Rapport structurel:")
import json
print(json.dumps(structural_report, indent=2, ensure_ascii=False))

In [ ]:

# Génération de la documentation technique

def generate_technical_documentation(description, params, structural_report):
    """Génère un rapport technique simple"""
    from datetime import datetime
    import json

    doc = {
        'projet': 'Dossier technique généré automatiquement',
        'date': datetime.now().isoformat(),
        'description_originale': description,
        'paramètres_extraits': params,
        'calculs_structurels': structural_report,
        'notes': 'Document généré automatiquement - Validation par un ingénieur recommandée'
    }

    # Sauvegarde en JSON
    with open('dossier_technique.json', 'w') as f:
        json.dump(doc, f, indent=2, ensure_ascii=False)

    # Conversion en PDF (simplifiée)
    try:
        from fpdf import FPDF
        pdf = FPDF()
        pdf.add_page()
        pdf.set_font("Arial", size=12)

        # Titre
        pdf.cell(200, 10, txt="DOSSIER TECHNIQUE", ln=True, align='C')
        pdf.ln(10)

        # Contenu
        pdf.multi_cell(0, 10, f"Date: {doc['date']}")
        pdf.multi_cell(0, 10, f"Description: {doc['description_originale']}")
        pdf.ln(5)

        pdf.multi_cell(0, 10, "Paramètres extraits:")
        for key, value in doc['paramètres_extraits'].items():
            pdf.multi_cell(0, 10, f" {key}: {value}")

        pdf.ln(5)
        pdf.multi_cell(0, 10, "Calculs structurels:")
        pdf.multi_cell(0, 10, json.dumps(doc['calculs_structurels'], indent=2, ensure_ascii=False))

        pdf.output("dossier_technique.pdf")
        print("Dossier technique PDF généré: dossier_technique.pdf")
    except ImportError:
        print("Installation de fpdf nécessaire pour la génération PDF: !pip install fpdf")

In [5]:
# Génération complète du dossier
generate_technical_documentation(description_test, params, structural_report)

NameError: name 'generate_technical_documentation' is not defined

In [6]:
#  Téléchargement des fichiers
try:
    from google.colab import files
    print("\nTéléchargement des fichiers générés:")
    files.download("output_plan.dxf")
    files.download("modele_3d.dxf")
    files.download("dossier_technique.json")

    # Essayer de télécharger le PDF s'il existe
    import os
    if os.path.exists("dossier_technique.pdf"):
        files.download("dossier_technique.pdf")
except:
    print("Téléchargement non disponible en dehors de Colab")


Téléchargement des fichiers générés:
Téléchargement non disponible en dehors de Colab
